# Linux Network

Ref: [Understanding Linux Network Internals]()


## chapter 1: Introduction

![](resources/01_01.png)
![](resources/01_02.png)

1. nice trick. `placeholder` takes 0 bytes memory.

## Chapter 2: Critical Data Structures

### sk_buff

[sk_buff previous notes](../notes4.4/network.ipynb)

主要有以下几点

1. 结构体内部对每一层都用union来包含各种protocol的header。这样一个结构体可以存储不同的protocol的packet
   
```c
union {
	struct tcphdr	*th;
	struct udphdr	*uh;
	struct icmphdr	*icmph;
	struct igmphdr	*igmph;
	struct iphdr	*ipiph;
	struct ipv6hdr	*ipv6h;
	unsigned char	*raw;
} h;

union {
	struct iphdr	*iph;
	struct ipv6hdr	*ipv6h;
	struct arphdr	*arph;
	unsigned char	*raw;
} nh;

union {
	unsigned char 	*raw;
} mac;
```

2. 结构体最后是指向真正的data buffer的一些指针

```c
/* These elements must be at the end, see alloc_skb() for details.  */
unsigned int    truesize;
atomic_t        users;
unsigned char	*head,*data,*tail,*end;
```
具体含义看前面的图。核心一点就是可以通过skb_reserve函数在buffer前面保留一段空间，方便后面加各个level的head

```c
static inline void skb_reserve(struct sk_buff *skb, unsigned int len)
{
    skb->data+=len;
    skb->tail+=len;
}
```

3. 处理packet的过程就是从data buffer里不断 解析/写入 各个level的数据，下图就是一个写入过程

![](resources/02_01.png)
